## Importar bibliotecas

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import requests
import re
import folium
import geopy
import ast
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
import haversine
#from sklearn.metrics.pairwise import haversine_distances
from math import radians
import time
import json
#import parser
from haversine import haversine
import pandas as pd
from pandas import json_normalize

from geopy.geocoders import Nominatim

#from bs4 import BeautifulSoup

import sys
sys.setrecursionlimit(2000)

# Show all Columns and Rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# bliblioteca = https://pypi.org/project/haversine/

print('Libraries imported')

Libraries imported


## Tratamento e consolidação de dados

### Informações das estações

In [3]:
station_dict = {'Catete' : '-22.925580460005982,-43.1766003091964',
                'Largo do Machado' : '-22.930651090814383,-43.17832974593375',
                'Flamengo' : '-22.93684861832403,-43.17829091934274',
                'Botafogo' : '-22.950622880463147,-43.18405212488933',
                'Cardeal Arco Verde' : '-22.96361994621739,-43.18083122492285',
                'Siqueira Campos' : '-22.966722295911186,-43.18649393180904',
                'Cantagalo' : '-22.976112641984226,-43.192580445299605',
                'General Osório' : '-22.984610664497772,-43.19718307781211'}
station_dict

{'Catete': '-22.925580460005982,-43.1766003091964',
 'Largo do Machado': '-22.930651090814383,-43.17832974593375',
 'Flamengo': '-22.93684861832403,-43.17829091934274',
 'Botafogo': '-22.950622880463147,-43.18405212488933',
 'Cardeal Arco Verde': '-22.96361994621739,-43.18083122492285',
 'Siqueira Campos': '-22.966722295911186,-43.18649393180904',
 'Cantagalo': '-22.976112641984226,-43.192580445299605',
 'General Osório': '-22.984610664497772,-43.19718307781211'}

In [4]:
station = [['Catete',-22.925580460005982,-43.1766003091964, 'Rio de Janeiro'],
           ['Largo do Machado',-22.930651090814383,-43.17832974593375, 'Rio de Janeiro'],
           ['Flamengo',-22.93684861832403,-43.17829091934274, 'Rio de Janeiro'],
           ['Botafogo',-22.950622880463147,-43.18405212488933, 'Rio de Janeiro'],
           ['Cardeal Arco Verde',-22.96361994621739,-43.18083122492285, 'Rio de Janeiro'],
           ['Siqueira Campos',-22.966722295911186,-43.18649393180904, 'Rio de Janeiro'],
           ['Cantagalo',-22.976112641984226,-43.192580445299605, 'Rio de Janeiro'],
           ['General Osório',-22.984610664497772,-43.19718307781211, 'Rio de Janeiro']]

titles =   ['estacao_nome','latitude','longitude','cidade']

df_station = pd.DataFrame(station, columns=titles)
df_station


,estacao_nome,latitude,longitude,cidade
0,Catete,-22.925580,-43.176600,Rio de Janeiro
1,Largo do Machado,-22.930651,-43.178330,Rio de Janeiro
2,Flamengo,-22.936849,-43.178291,Rio de Janeiro
3,Botafogo,-22.950623,-43.184052,Rio de Janeiro
4,Cardeal Arco Verde,-22.963620,-43.180831,Rio de Janeiro
5,Siqueira Campos,-22.966722,-43.186494,Rio de Janeiro
6,Cantagalo,-22.976113,-43.192580,Rio de Janeiro
7,General Osório,-22.984611,-43.197183,Rio de Janeiro


### Tratamento das Categorias

#### Abrir planilhas de categorias

In [5]:
#academia
academia = 'C:/Users/renan/OneDrive/Área de Trabalho/Estudos/Infnet/Notebooks/Projeto_z_sul/handling_academia_ginasio_estudio.csv'
df_academia = pd.read_csv(academia, index_col=False)

#cafeteria
cafeteria = 'C:/Users/renan/OneDrive/Área de Trabalho/Estudos/Infnet/Notebooks/Projeto_z_sul/handling_cafeteria.csv'
df_cafeteria = pd.read_csv(cafeteria)

#educacao
educacao = 'C:/Users/renan/OneDrive/Área de Trabalho/Estudos/Infnet/Notebooks/Projeto_z_sul/handling_educacao.csv'
df_educacao = pd.read_csv(educacao)

#hotel
hotel = 'C:/Users/renan/OneDrive/Área de Trabalho/Estudos/Infnet/Notebooks/Projeto_z_sul/handling_hotel.csv'
df_hotel = pd.read_csv(hotel)

#padaria
padaria = 'C:/Users/renan/OneDrive/Área de Trabalho/Estudos/Infnet/Notebooks/Projeto_z_sul/handling_padaria.csv'
df_padaria = pd.read_csv(padaria)

#restaurante
restaurante = 'C:/Users/renan/OneDrive/Área de Trabalho/Estudos/Infnet/Notebooks/Projeto_z_sul/handling_restaurante.csv'
df_restaurante = pd.read_csv(restaurante)

#teat_cin
teat_cin = 'C:/Users/renan/OneDrive/Área de Trabalho/Estudos/Infnet/Notebooks/Projeto_z_sul/handling_teatro_e_cinema.csv'
df_teat_cin = pd.read_csv(teat_cin)

#### Juntar planilhas de categorias e calcular distância dos estabelecimentos para o metrô.

In [6]:
# Dataframes por importância:
df_append = [df_padaria, df_cafeteria, df_restaurante, df_academia, df_educacao, df_hotel, df_teat_cin]
df_estab = pd.DataFrame()
for df in df_append:
    df_estab = df_estab.append(df)
df_estab.drop(df_estab.columns[[0]],axis=1, inplace=True)

In [7]:
df_estab.head(1)

,name,estacao_nome,estacao_lat_lon,endereço,latitude,longitude,estabelecimento
0,La Veronese,General Osório,"-22.984610664497772,-43.19718307781211","Rua Visconde de Piraja, 29, Rio de Janeiro, RJ...",-22.984827,-43.19636,padaria


In [8]:
df_estab['estacao_lat'], df_estab['estacao_lon'] = df_estab['estacao_lat_lon'].str.split(',', 1).str
del df_estab['estacao_lat_lon']

In [9]:
df_estab = df_estab[['name','endereço','estabelecimento','latitude','longitude','estacao_nome','estacao_lat','estacao_lon']]

In [10]:
df_estab.head(2)

,name,endereço,estabelecimento,latitude,longitude,estacao_nome,estacao_lat,estacao_lon
0,La Veronese,"Rua Visconde de Piraja, 29, Rio de Janeiro, RJ...",padaria,-22.984827,-43.19636,General Osório,-22.984610664497772,-43.19718307781211
1,Padaria Santa Marta,"Avenida Epitácio Pessoa, 1789, Rio de Janeiro, RJ",padaria,-22.978994,-43.20080,General Osório,-22.984610664497772,-43.19718307781211


In [11]:
df_estab.shape

(2182, 8)

In [12]:
df_estab.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2182 entries, 0 to 166
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             2182 non-null   object 
 1   endereço         2089 non-null   object 
 2   estabelecimento  2182 non-null   object 
 3   latitude         2182 non-null   float64
 4   longitude        2182 non-null   float64
 5   estacao_nome     2182 non-null   object 
 6   estacao_lat      2182 non-null   object 
 7   estacao_lon      2182 non-null   object 
dtypes: float64(2), object(6)
memory usage: 153.4+ KB


In [13]:
df_estab['estacao_lat'] = df_estab['estacao_lat'].astype(float)
df_estab['estacao_lon'] = df_estab['estacao_lon'].astype(float)

In [14]:
distancia = []
df_test = df_estab[['latitude', 'longitude', 'estacao_lat', 'estacao_lon']]
for idx, row in df_test.iterrows():
    distancia.append(haversine(row[0:2], row[2:4], unit='m'))

In [15]:
df_estab['distancia'] = distancia
df_estab['distancia'] = df_estab['distancia'].astype(int)

In [16]:
df_estab.shape

(2182, 9)

In [17]:
df_estab = df_estab.sort_values("distancia", ascending = True).drop_duplicates(subset=['name','endereço','latitude','longitude'], keep="first")

In [28]:
df_estab.shape

(1169, 9)

In [19]:
df_estab.sort_index(ascending=True, inplace=True)

In [20]:
df_estab.head()

,name,endereço,estabelecimento,latitude,longitude,estacao_nome,estacao_lat,estacao_lon,distancia
0,Brasas,"Rua Pompeu Loureiro, 41, Rio de Janeiro, RJ, 2...",educação,-22.973412,-43.19220,General Osório,-22.984611,-43.197183,1345
0,Academia do Tap,"Rua Visconde de Pirajá, 235, Rio de Janeiro, R...",academia_ginasio_estudio,-22.984525,-43.20231,General Osório,-22.984611,-43.197183,524
0,La Veronese,"Rua Visconde de Piraja, 29, Rio de Janeiro, RJ...",padaria,-22.984827,-43.19636,General Osório,-22.984611,-43.197183,87
0,Mercearia da Praça,"Rua Jangadeiros, 28, Ljs. A/B (Pç. Gen. Osório...",restaurante,-22.984848,-43.19680,General Osório,-22.984611,-43.197183,47
0,Hotel Fasano Rio de Janeiro,"Avenida Vieira Souto, 80, Rio de Janeiro, RJ, ...",hotel,-22.987191,-43.19624,General Osório,-22.984611,-43.197183,302


In [25]:
df_estab.groupby(['estabelecimento','estacao_nome'])['estacao_nome'].count()

estabelecimento           estacao_nome      
academia_ginasio_estudio  Botafogo              44
                          Cantagalo             20
                          Cardeal Arco Verde    23
                          Catete                20
                          Flamengo              25
                          General Osório        44
                          Largo do Machado      21
                          Siqueira Campos       19
cafeteria                 Botafogo               4
                          Cardeal Arco Verde     2
                          Catete                 1
                          Flamengo               2
educação                  Botafogo              38
                          Cantagalo             29
                          Cardeal Arco Verde    25
                          Catete                32
                          Flamengo              31
                          General Osório        42
                          Largo do Ma

In [26]:
df_estab.groupby(['estacao_nome','estabelecimento'])['estabelecimento'].count()

estacao_nome        estabelecimento         
Botafogo            academia_ginasio_estudio    44
                    cafeteria                    4
                    educação                    38
                    hotel                       36
                    padaria                     44
                    restaurante                 42
                    teatro_e_cinema             25
Cantagalo           academia_ginasio_estudio    20
                    educação                    29
                    hotel                       18
                    padaria                     22
                    restaurante                 19
                    teatro_e_cinema              3
Cardeal Arco Verde  academia_ginasio_estudio    23
                    cafeteria                    2
                    educação                    25
                    hotel                       28
                    padaria                     16
                    restaurante      

In [27]:
df_estab[['name','estacao_nome',]].groupby(['name']) ['estacao_nome'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(5)

,name,count
989,Smart Fit,15
201,Bodytech,7
8,Academia,5
644,Kinoplex,4
210,Boteco Belmonte,4


In [29]:
df_estab.to_csv("2_not_agrup_trat.csv")